# MLQA Extract `English` Keywords (Query/Context)

In [2]:
import os
import re
import json
from tqdm.auto import tqdm
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForTokenClassification

from torch.utils.data import DataLoader, Dataset
import torch
from torch.nn.functional import softmax

In [ ]:
file_path = '/kaggle/input/m-l-q-a/Context_EN.json'

with open(file_path, 'r') as f:
    data = json.load(f)

print(f"Total num: {len(data)}")
data[:1]

Total num: 9916


[{'id': 0,
  'title': 'Area 51',
  'context': 'In 1994, five unnamed civilian contractors and the widows of contractors Walter Kasza and Robert Frost sued the USAF and the United States Environmental Protection Agency. Their suit, in which they were represented by George Washington University law professor Jonathan Turley, alleged they had been present when large quantities of unknown chemicals had been burned in open pits and trenches at Groom. Biopsies taken from the complainants were analyzed by Rutgers University biochemists, who found high levels of dioxin, dibenzofuran, and trichloroethylene in their body fat. The complainants alleged they had sustained skin, liver, and respiratory injuries due to their work at Groom, and that this had contributed to the deaths of Frost and Kasza. The suit sought compensation for the injuries they had sustained, claiming the USAF had illegally handled toxic materials, and that the EPA had failed in its duty to enforce the Resource Conservation an

# BART method (Query)

**Set up BART**

In [ ]:
class TextDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

tokenizer = AutoTokenizer.from_pretrained("Andyrasika/bart_tech_keywords")
model = AutoModelForSeq2SeqLM.from_pretrained("Andyrasika/bart_tech_keywords")

'''
    * For "Question", set `texts` with `[item['question'] for item in data]`
    * For "Context", set `texts` with `[item['context'] for item in data]`
'''
texts = [item['context'] for item in data]

encodings = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")

# Use dataloader
dataset = TextDataset(encodings)
dataloader = DataLoader(dataset, batch_size=16)

tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/292 [00:00<?, ?B/s]

In [ ]:
model.to("cuda")
text_outputs = []

for batch in tqdm(dataloader, desc="Generating text"):
    batch = {k: v.to("cuda") for k, v in batch.items()} 
    with torch.no_grad():
        outputs = model.generate(**batch)
        decoded_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
        text_outputs.extend(decoded_texts)

Generating text:   0%|          | 0/620 [00:00<?, ?it/s]

/tmp/ipykernel_34/2267505483.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1128: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


**Save results**

In [ ]:
assert len(texts) == len(text_outputs)
len(texts), len(text_outputs)

(9916, 9916)

In [ ]:
strip_key_words = [line.strip() for line in text_outputs]

'''
    * For "Question", set `ids` with `item['id']`
    * For "Context", set `ids` with `item['context_id']`
'''
ids = [item['id'] for item in data]
titles = [item['title'] for item in data]

assert len(ids) == len(strip_key_words)
assert len(ids) == len(titles)

print(f"Total num: {len(strip_key_words)}")

Total num: 9916


In [ ]:
documents = [
    {
        "id": id, 
        "title": title,
        "keywords": [k.strip() for k in kw.split(',') if k.strip()]  # Strip and remove empty values
    } 
    for id, title, kw in zip(ids, titles, strip_key_words)
]

json_data = json.dumps(documents, indent=4)


In [ ]:
with open('/kaggle/working/Context_BART_Keywords_MLQA_en.json', 'w') as f:
    f.write(json_data)

# with open('./data/BART_Keywords.json', 'w') as f:
#     f.write(json_data)

## Artificial add prob

In [ ]:
file_path_ = '/kaggle/working/Query_BART_Keywords_MLQA_en.json'

with open(file_path_, 'r') as f:
    Query_BART_Keywords = json.load(f)

print(f"Total num: {len(Query_BART_Keywords)}")
Query_BART_Keywords[:5]

Total num: 11590


[{'id': 'a4968ca8a18de16aa3859be760e43dbd3af3fce9',
  'keywords': ['Biopsies', 'Analysis']},
 {'id': 'f251ea56c4f1aa1df270137f7e6d89c0cc1b6ef4',
  'keywords': ['Robert Frost', 'Walter Kasza', 'Lawsuit']},
 {'id': '04ecd5555635bc05fd2f379d1b9027edd663cebf',
  'keywords': ['Lawsuit', 'Groom']},
 {'id': 'd066a75dbe8cd3e2b57c415a8eb54a08dc7e72a7',
  'keywords': ['Complaints', 'Allegations']},
 {'id': 'c5f545baccd8ea8adb83f75756f4832340600bd9',
  'keywords': ['Aerospace Magazine']}]

In [ ]:
for item in Query_BART_Keywords:
    num_keywords = len(item["keywords"])
    probs = [max(0.9 - 0.1 * i, 0.1) for i in range(num_keywords)]
    item["prob"] = probs

Query_BART_Keywords[:5]

[{'id': 'a4968ca8a18de16aa3859be760e43dbd3af3fce9',
  'keywords': ['Biopsies', 'Analysis'],
  'prob': [0.9, 0.8]},
 {'id': 'f251ea56c4f1aa1df270137f7e6d89c0cc1b6ef4',
  'keywords': ['Robert Frost', 'Walter Kasza', 'Lawsuit'],
  'prob': [0.9, 0.8, 0.7]},
 {'id': '04ecd5555635bc05fd2f379d1b9027edd663cebf',
  'keywords': ['Lawsuit', 'Groom'],
  'prob': [0.9, 0.8]},
 {'id': 'd066a75dbe8cd3e2b57c415a8eb54a08dc7e72a7',
  'keywords': ['Complaints', 'Allegations'],
  'prob': [0.9, 0.8]},
 {'id': 'c5f545baccd8ea8adb83f75756f4832340600bd9',
  'keywords': ['Aerospace Magazine'],
  'prob': [0.9]}]

In [ ]:
json_data = json.dumps(Query_BART_Keywords, indent=4)

with open('/kaggle/working/Query_BART_Keywords_artificially_prob_MLQA_en.json', 'w') as f:
    f.write(json_data)

### **Add `title` to `query`**

#### **For `Query_BART_Keywords_MLQA_en.json`**

In [ ]:
file_path = './data/QA_EN.json'

with open(file_path, 'r') as f:
    data_en_qa = json.load(f)

print(f"Total num: {len(data_en_qa)}")
data_en_qa[:2]

Total num: 11590


[{'question': 'Who analyzed the biopsies?',
  'answers': [{'text': 'Rutgers University biochemists', 'answer_start': 457}],
  'id': 'a4968ca8a18de16aa3859be760e43dbd3af3fce9',
  'context_id': 0,
  'title': 'Area 51'},
 {'question': 'who represented robert frost and walter kasza in their suit?',
  'answers': [{'text': 'George Washington University law professor Jonathan Turley',
    'answer_start': 218}],
  'id': 'f251ea56c4f1aa1df270137f7e6d89c0cc1b6ef4',
  'context_id': 0,
  'title': 'Area 51'}]

In [ ]:
file_path = './data/MLQA_Keywords/en/Query_BART_Keywords_MLQA_en.json'
# file_path = './data/MLQA_Keywords/en/Query_BART_Keywords_artificially_prob_MLQA_en.json'
# file_path = './data/MLQA_Keywords/en/Query_BART_Keywords_with_prob_MLQA_en.json'

# file_path = './data/MLQA_Keywords/en/Context_with_title_BART_Keywords_MLQA_en.json'

with open(file_path, 'r') as f:
    query_kw = json.load(f)
print(f"Total num: {len(query_kw)}")
query_kw[:2]

Total num: 11590


[{'id': 'a4968ca8a18de16aa3859be760e43dbd3af3fce9',
  'keywords': ['Biopsies', 'Analysis']},
 {'id': 'f251ea56c4f1aa1df270137f7e6d89c0cc1b6ef4',
  'keywords': ['Robert Frost', 'Walter Kasza', 'Lawsuit']}]

In [ ]:
ids = [item['id'] for item in data_en_qa]
context_ids = [item['context_id'] for item in data_en_qa]
titles = [item['title'] for item in data_en_qa]
# Clean title
# titles = [re.sub(r'\W+', '', title) for title in titles]
titles = [re.sub(r'[^\w\s]+', '', title) for title in titles]

keywords_list = [item['keywords'] for item in query_kw]
# prob_list = [item['prob'] for item in query_kw]

assert len(ids_list) == len(keywords_list)
# assert len(ids_list) == len(prob_list)


strip_key_words = [([titles[i]] + sublist) for i, sublist in enumerate(keywords_list)]

assert len(ids) == len(strip_key_words)

print(f"Total num: {len(strip_key_words)}")


Total num: 11590


**Drop duplicate title and keywords**

In [ ]:
unique_data = [list(dict.fromkeys(sublist)) for sublist in strip_key_words]

assert len(unique_data) == len(strip_key_words)
print(f"Total num: {len(unique_data)}")

Total num: 11590


In [ ]:
documents = [
    {   
        "id": id, 
        "context id": c_id,
        "keywords": kw
    } 
    for id, c_id, kw in zip(ids, context_ids, unique_data)
]

json_data = json.dumps(documents, indent=4)

In [ ]:
with open('./data/MLQA_Keywords/en/Query_BART_Keywords_with_title_MLQA_en.json', 'w') as f:
    f.write(json_data)

In [ ]:
file_path = './data/MLQA_Keywords/en/Query_BART_Keywords_with_title_MLQA_en.json'

with open(file_path, 'r') as f:
    Query_BART_Keywords = json.load(f)
Query_BART_Keywords[:2]

[{'id': 'a4968ca8a18de16aa3859be760e43dbd3af3fce9',
  'context id': 0,
  'keywords': ['Area 51', 'Biopsies', 'Analysis']},
 {'id': 'f251ea56c4f1aa1df270137f7e6d89c0cc1b6ef4',
  'context id': 0,
  'keywords': ['Area 51', 'Robert Frost', 'Walter Kasza', 'Lawsuit']}]

#### **For `Query_BART_Keywords_artificially_prob_MLQA_en.json`**

**Add prob**

In [ ]:
for item in Query_BART_Keywords:
    num_keywords = len(item["keywords"])
    probs = [max(0.9 - 0.1 * i, 0.1) for i in range(num_keywords)]
    item["prob"] = probs

Query_BART_Keywords[:5]

[{'id': 'a4968ca8a18de16aa3859be760e43dbd3af3fce9',
  'context id': 0,
  'keywords': ['Area 51', 'Biopsies', 'Analysis'],
  'prob': [0.9, 0.8, 0.7]},
 {'id': 'f251ea56c4f1aa1df270137f7e6d89c0cc1b6ef4',
  'context id': 0,
  'keywords': ['Area 51', 'Robert Frost', 'Walter Kasza', 'Lawsuit'],
  'prob': [0.9, 0.8, 0.7, 0.6]},
 {'id': '04ecd5555635bc05fd2f379d1b9027edd663cebf',
  'context id': 0,
  'keywords': ['Area 51', 'Lawsuit', 'Groom'],
  'prob': [0.9, 0.8, 0.7]},
 {'id': 'd066a75dbe8cd3e2b57c415a8eb54a08dc7e72a7',
  'context id': 0,
  'keywords': ['Area 51', 'Complaints', 'Allegations'],
  'prob': [0.9, 0.8, 0.7]},
 {'id': 'c5f545baccd8ea8adb83f75756f4832340600bd9',
  'context id': 1,
  'keywords': ['Area 51', 'Aerospace Magazine'],
  'prob': [0.9, 0.8]}]

In [ ]:
json_data = json.dumps(Query_BART_Keywords, indent=4)

with open('./data/MLQA_Keywords/en/Query_BART_Keywords_with_artificially_prob_MLQA_en.json', 'w') as f:
    f.write(json_data)

## Bart generate prob

**Set model to evaluate mode**

In [ ]:
model.eval()
model.to("cuda")

all_input_texts = []
all_generated_texts = []
all_probabilities = []

for batch in tqdm(dataloader):
    input_ids = batch['input_ids'].to(model.device)
    attention_mask = batch['attention_mask'].to(model.device)

    for idx in range(input_ids.size(0)):  # Iter all in a batch
        decoded_ids = torch.full((1, 1), tokenizer.bos_token_id, dtype=torch.long, device=model.device)
        seq_probabilities = []

        input_text = tokenizer.decode(input_ids[idx].tolist(), skip_special_tokens=True)
        all_input_texts.append(input_text) 
        
        while True:
            with torch.no_grad():
                outputs = model(input_ids[idx:idx+1], attention_mask=attention_mask[idx:idx+1], decoder_input_ids=decoded_ids)
                logits = outputs.logits[:, -1, :]
                probs = softmax(logits, dim=-1)
                next_token_id = torch.argmax(probs, dim=-1).unsqueeze(-1)
                next_token_prob = probs[0, next_token_id.item()].item()

                if next_token_id == tokenizer.eos_token_id or decoded_ids.size(1) >= 30:
                    break

                decoded_ids = torch.cat([decoded_ids, next_token_id], dim=-1)
                seq_probabilities.append(next_token_prob)

        generated_text = tokenizer.decode(decoded_ids[:, 1:].squeeze().tolist(), skip_special_tokens=True)

        # Splitting the generated text and probability
        segments = generated_text.split(',')
        segment_probs = []
        start_idx = 0
        for segment in segments:
            segment_tokens = tokenizer.tokenize(segment.strip())
            segment_length = len(segment_tokens)
            # Calc avg prob of the segment
            if segment_length > 0:
                segment_prob = sum(seq_probabilities[start_idx:start_idx+segment_length]) / segment_length
                segment_probs.append(f"{segment_prob:.6f}")
            start_idx += segment_length

        all_generated_texts.append(generated_text)
        all_probabilities.append(",".join(segment_probs))

  0%|          | 0/725 [00:00<?, ?it/s]

/tmp/ipykernel_34/4251668379.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [ ]:
len(all_generated_texts), len(all_probabilities)

(11590, 11590)

In [ ]:
strip_key_words = [line.strip() for line in all_generated_texts]

'''
    * For "Question", set `ids` with `[item['id'] for item in data]`
    * For "Context", set `ids` with `[item['context_id'] for item in data]`
'''

ids = [item['id'] for item in data_en_qa]
context_ids = [item['context_id'] for item in data_en_qa]

len(ids),len(strip_key_words)

print(f"Total num: {len(strip_key_words)}")

Total num: 11590


**Output `WITHOUT title` version**

In [ ]:
documents = [
    {
        "id": id, 
        "context id": c_id,
        "keywords": [k.strip() for k in kw.split(',')],
        "prob": [float(p) for p in prob.split(',')]
    } 
    for id, c_id, kw, prob in zip(ids, context_ids, strip_key_words, all_probabilities)
    if kw.strip() != '' and prob.strip() != ''
]

json_data = json.dumps(documents, indent=4)


In [ ]:
with open('/kaggle/working/Query_BART_Keywords_with_prob_MLQA_en.json', 'w') as f:
    f.write(json_data)

In [ ]:
file_path_ = '/kaggle/working/Query_BART_Keywords_with_prob_MLQA_en.json'

with open(file_path_, 'r') as f:
    data_ = json.load(f)

print(f"Total num: {len(data_)}")
data_[:5]

Total num: 11110


[{'id': 'a4968ca8a18de16aa3859be760e43dbd3af3fce9',
  'context id': 0,
  'keywords': ['Bi', 'Biopsies'],
  'prob': [0.80947, 0.553737]},
 {'id': 'f251ea56c4f1aa1df270137f7e6d89c0cc1b6ef4',
  'context id': 0,
  'keywords': ['RobertKasza', 'Frost', 'Lawsuit'],
  'prob': [0.673948, 0.663587, 0.753613]},
 {'id': '04ecd5555635bc05fd2f379d1b9027edd663cebf',
  'context id': 0,
  'keywords': ['Groom', 'Lawsuit'],
  'prob': [0.424797, 0.579578]},
 {'id': 'd066a75dbe8cd3e2b57c415a8eb54a08dc7e72a7',
  'context id': 0,
  'keywords': ['Complaintants', 'alleged'],
  'prob': [0.639551, 0.296297]},
 {'id': 'c5f545baccd8ea8adb83f75756f4832340600bd9',
  'context id': 1,
  'keywords': ['A Aerospace Magazine'],
  'prob': [0.599785]}]

**Output `with title` version**


In [ ]:
ids = [item['id'] for item in data_en_qa]
context_ids = [item['context_id'] for item in data_en_qa]
titles = [item['title'] for item in data_en_qa]
# Clean title
# titles = [re.sub(r'\W+', '', title) for title in titles]
titles = [re.sub(r'[^\w\s]+', '', title) for title in titles]

keywords_list = [item.split(', ') for item in strip_key_words]
prob_list = [
    [float(num) if num != '' else 0.0 for num in item.split(',')] 
    for item in all_probabilities
]

assert len(ids) == len(keywords_list)
assert len(ids) == len(prob_list)

updated_keywords_list = []
updated_prob_list = []

for i, (sublist, prob_sublist) in enumerate(zip(keywords_list, prob_list)):
    # If the keywords sublist is empty
    if not sublist:  
        # Use title as the keyword
        sublist = [titles[i]]  
        prob_sublist = [0.99]
    else:
        # Ensure new_prob does not exceed 1
        new_prob = min(prob_sublist[0] + 0.1, 1.0)  
        if titles[i] in sublist:
            # Remove duplicate keyword and corresponding probability
            index = sublist.index(titles[i])
            del sublist[index]  
            del prob_sublist[index]
        # Add title to head of keywords
        sublist = [titles[i]] + sublist  
        prob_sublist = [new_prob] + prob_sublist
    
    updated_keywords_list.append(sublist)
    updated_prob_list.append(prob_sublist)

assert len(updated_prob_list) == len(updated_keywords_list)
print(f"Total num: {len(updated_keywords_list)}")

Total num: 11590


In [ ]:
documents = [
    {   
        "id": id, 
        "context id": c_id,
        "keywords": kw,
        "prob": p,
    } 
    for id, c_id, kw, p in zip(ids, context_ids, updated_keywords_list, updated_prob_list)
]

json_data = json.dumps(documents, indent=4)

with open('/kaggle/working/Query_BART_Keywords_with_prob_and_title_MLQA_en.json', 'w') as f:
    f.write(json_data)


In [ ]:
file_path_ = '/kaggle/working/Query_BART_Keywords_with_prob_and_title_MLQA_en.json'

with open(file_path_, 'r') as f:
    data_ = json.load(f)

print(f"Total num: {len(data_)}")
data_[:5]

Total num: 11590


[{'id': 'a4968ca8a18de16aa3859be760e43dbd3af3fce9',
  'context id': 0,
  'keywords': ['Area 51', 'Bi', 'Biopsies'],
  'prob': [0.90947, 0.80947, 0.553737]},
 {'id': 'f251ea56c4f1aa1df270137f7e6d89c0cc1b6ef4',
  'context id': 0,
  'keywords': ['Area 51', 'RobertKasza', 'Frost', 'Lawsuit'],
  'prob': [0.773948, 0.673948, 0.663587, 0.753613]},
 {'id': '04ecd5555635bc05fd2f379d1b9027edd663cebf',
  'context id': 0,
  'keywords': ['Area 51', 'Groom', 'Lawsuit'],
  'prob': [0.524797, 0.424797, 0.579578]},
 {'id': 'd066a75dbe8cd3e2b57c415a8eb54a08dc7e72a7',
  'context id': 0,
  'keywords': ['Area 51', 'Complaintants', 'alleged'],
  'prob': [0.739551, 0.639551, 0.296297]},
 {'id': 'c5f545baccd8ea8adb83f75756f4832340600bd9',
  'context id': 1,
  'keywords': ['Area 51', 'A Aerospace Magazine'],
  'prob': [0.699785, 0.599785]}]

## For Context **Add `title` to `query`**

In [70]:
file_path = './data/Context_EN.json'

with open(file_path, 'r') as f:
    data_en = json.load(f)

print(f"Total num: {len(data_en)}")
data_en[:2]

Total num: 9916


[{'id': 0,
  'title': 'Area 51',
  'context': 'In 1994, five unnamed civilian contractors and the widows of contractors Walter Kasza and Robert Frost sued the USAF and the United States Environmental Protection Agency. Their suit, in which they were represented by George Washington University law professor Jonathan Turley, alleged they had been present when large quantities of unknown chemicals had been burned in open pits and trenches at Groom. Biopsies taken from the complainants were analyzed by Rutgers University biochemists, who found high levels of dioxin, dibenzofuran, and trichloroethylene in their body fat. The complainants alleged they had sustained skin, liver, and respiratory injuries due to their work at Groom, and that this had contributed to the deaths of Frost and Kasza. The suit sought compensation for the injuries they had sustained, claiming the USAF had illegally handled toxic materials, and that the EPA had failed in its duty to enforce the Resource Conservation an

In [72]:
file_path_ = './data/MLQA_Keywords/en/Context_with_title_BART_Keywords_MLQA_en.json'

with open(file_path_, 'r') as f:
    data_ = json.load(f)

print(f"Total num: {len(data_)}")
data_[:3]

Total num: 9916


[{'id': 0,
  'title': 'Area 51',
  'keywords': ['Civilian Contractors',
   'USAF',
   'Environmental Protection Agency',
   'Area 51',
   'Toxic Materials']},
 {'id': 1,
  'title': 'Area 51',
  'keywords': ['Astronauts', 'Area 51', 'Skylab Incident']},
 {'id': 2,
  'title': 'Area 51',
  'keywords': ['Area 51', 'UFO', 'Conspiracy Theories']}]

In [80]:
ids = [item['id'] for item in data_]
kws = [item['keywords'] for item in data_]
titles = [item['title'] for item in data_]
titles = [re.sub(r'[^\w\s]+', '', title) for title in titles]

assert len(ids) == len(titles)
assert len(ids) == len(kws)


strip_key_words = [([titles[i]] + sublist) for i, sublist in enumerate(kws)]

assert len(ids) == len(strip_key_words)

print(f"Total num: {len(strip_key_words)}")

unique_data = [list(dict.fromkeys(sublist)) for sublist in strip_key_words]

assert len(unique_data) == len(strip_key_words)
print(f"Total num: {len(unique_data)}")

Total num: 9916
Total num: 9916


In [81]:
documents = [
    {   
        "id": id, 
        "keywords": kw
    } 
    for id, kw in zip(ids, unique_data)
]

json_data = json.dumps(documents, indent=4)

with open('./data/MLQA_Keywords/en/Context_with_title_BART_Keywords_MLQA_en.json', 'w') as f:
    f.write(json_data)

In [82]:
file_path_ = './data/MLQA_Keywords/en/Context_with_title_BART_Keywords_MLQA_en.json'

with open(file_path_, 'r') as f:
    data_ = json.load(f)

print(f"Total num: {len(data_)}")
data_[:3]

Total num: 9916


[{'id': 0,
  'keywords': ['Area 51',
   'Civilian Contractors',
   'USAF',
   'Environmental Protection Agency',
   'Toxic Materials']},
 {'id': 1, 'keywords': ['Area 51', 'Astronauts', 'Skylab Incident']},
 {'id': 2, 'keywords': ['Area 51', 'UFO', 'Conspiracy Theories']}]